## Loading packages

In [1]:
import numpy as np
import pandas as pd
import math
import itertools
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from scipy import stats

from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.callbacks import Callback, ModelCheckpoint
from keras import backend as K

from tensorflow.keras import layers  
import tensorflow as tf
tf.random.set_seed(2) 

from numpy.random import seed
seed(1)

## Defining the functions


In [2]:
###create model
def create_model(optimizer, activation):
    
    ###define model
    model = Sequential()
    model.add(Dense(512, activation=activation, input_shape=(X.shape[1],)))  ## to be changed as per the organism's features
    model.add(Dense(256, activation=activation))
    model.add(Dense(128, activation=activation))
    model.add(Dense(64, activation=activation))
    model.add(Dense(32, activation=activation))
    model.add(Dense(16, activation=activation))
    model.add(Dense(8, activation=activation))
    model.add(Dense(1))
        
    ###compile model
    model.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['mae'])
    
    return model


###fit model
def fit_model(X_train, y_train, X_test, y_test,n,model_path,model):
    ###define checkpoint for the best model
    checkpoint = ModelCheckpoint(filepath=model_path, verbose=1, monitor='val_loss',save_best_only=False, mode='max')
    ###fit model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=n, validation_split=0.1, verbose=0, callbacks=[checkpoint])


###predict the independent validation set
def predict_validation(X, y, model_path,model,optimizer):
    model.load_weights(model_path)
    model.compile(loss='mean_absolute_error',optimizer=optimizer)
    y_pred = model.predict(X)
    result = measurements(y,  y_pred)
    return result


In [3]:
def measurements(y_test, y_pred):
    
    mse=mean_squared_error(y_test,y_pred)
    rmse=np.sqrt(mse)
    r2=metrics.r2_score(y_test,y_pred)    
    pcc=stats.spearmanr(y_test,y_pred)

    return [mse, rmse, r2, pcc[0],pcc[1]]


In [4]:
def print_result(model_name, purpose, result):
    print('\033[1mOptimized {} model {} performance: \033[0m'.format(model_name, purpose))
    print("MSE:         {0:.3f}".format(result[0]))
    print("RMSE: {0:.3f}".format(result[1]))
    print("R2: {0:.3f}".format(result[2]))
    print("PCC:    {0:.3f}".format(result[3]))
    print("p-value:    {0:.3f}".format(result[4]))

 ## Data import

##### Training dataset

In [5]:
## Read the training dataset
overall_data =pd.read_csv("../overall_data/fs/overall_50_SVR_input_final_train.csv", low_memory=False)

## Remove last column from the training dataset
overall_data = overall_data.iloc[:, :-1]


## Replace the string character from the training dataset
overall_data.replace('?',0,inplace=True)

# overall_data.replace([np.inf, -np.inf], np.nan, inplace=True)
# overall_data.fillna(0, inplace=True)

X = overall_data.iloc[:,1:].values
y = overall_data.loc[:,'Name'].values

X = StandardScaler().fit(X).transform(X)

X = np.asarray(X).astype(np.float64)
y = np.asarray(y).astype(np.float64)

X[:] = np.nan_to_num(X)
y[:] = np.nan_to_num(y)

print(overall_data.shape)
overall_data.head()

(372, 51)


,Name,GraphFP762,THSA,ExtFP329,SubFP151,FP397,FP226,KRFPC439,L3m,KRFP4259,...,ExtFP383,RDF95u,LOBMAX,RDF50i,LOBMIN,ETA_Shape_Y,ExtFP364,RDF105u,RDF50u,RDF60s
0,7.85,0,795.591552,1,0.0,0,1,0.0,1.496764,0,...,0,23.912348,2.254889,80.686974,2.254889,0.38477,0,22.602817,63.952108,42.590737
1,5.20,0,619.994096,0,0.0,0,1,0.0,0.416425,0,...,0,14.038770,2.296008,42.595208,2.296008,0.26571,0,11.525543,32.399142,13.251245
2,5.08,0,564.571231,0,0.0,0,1,0.0,0.893505,0,...,0,10.962700,1.787919,42.030436,1.787919,0.25783,0,8.241788,32.564356,11.338215
3,5.92,0,389.414612,1,0.0,0,0,0.0,0.400007,0,...,0,1.687871,1.735348,18.061649,1.735348,0.38134,1,0.739854,12.289802,20.150016
4,4.99,0,701.759215,1,0.0,0,0,0.0,1.032301,0,...,0,15.051682,2.359914,38.534532,2.359914,0.24757,0,13.285121,30.935544,21.613635


##### Testing dataset

In [6]:

## Read the testing validation dataset
test = pd.read_csv("../overall_data/fs/overall_50_SVR_input_final_test.csv", low_memory=False)

## Remove last column from the test dataset
test = test.iloc[:, :-1]

## Replace the string character from the test dataset
test.replace('?',0,inplace=True)


X_val = test.iloc[:,1:].values
y_val = test.loc[:,'Name'].values

X_val = np.asarray(X_val).astype(np.float64)
y_val = np.asarray(y_val).astype(np.float64)

X_val[:] = np.nan_to_num(X_val)
y_val[:] = np.nan_to_num(y_val)


print(test.shape)
test.head()

(42, 51)


,Name,GraphFP762,THSA,ExtFP329,SubFP151,FP397,FP226,KRFPC439,L3m,KRFP4259,...,ExtFP383,RDF95u,LOBMAX,RDF50i,LOBMIN,ETA_Shape_Y,ExtFP364,RDF105u,RDF50u,RDF60s
0,6.72,0,392.171621,0,0,0,0,0.0,1.382606,0,...,0,5.455623,1.532245,30.045451,1.532245,0.43321,0,2.424224e-04,23.139069,17.965843
1,4.78,0,642.085448,1,0,0,0,0.0,0.993780,0,...,0,6.193387,1.435826,35.255120,1.282884,0.30267,0,3.252532e+00,28.020634,15.057649
2,5.03,0,687.487254,0,0,0,0,0.0,0.803854,0,...,0,27.364599,1.925171,40.872244,1.925171,0.27938,0,2.194966e+01,33.429509,19.092388
3,4.93,0,542.311934,0,0,0,0,0.0,0.746397,0,...,0,1.161963,1.375537,38.579777,1.375537,0.32566,0,2.931543e-25,31.694768,13.400968
4,4.90,0,617.888509,1,0,0,0,0.0,0.865613,0,...,0,6.250879,1.708695,37.344282,1.708695,0.18186,0,1.414258e+00,29.677617,15.234990


## Parameter tunning

In [7]:
### Set the optimizers and activators

# optimizers = ['SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
# activations = ['relu', 'sigmoid', 'softmax', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential']

optimizers = ['Adam', 'RMSprop','SGD', 'RMSprop']
activations = [ 'tanh', 'sigmoid', 'softmax']

# optimizers = ['Adam', ]
# activations = [ 'sigmoid']


paras = [' '.join(l) for l in itertools.product(optimizers, activations)]

col_names =  ['Range_val', 'Model_info','MAE', 'RMSE','R2', 'PCC','p-val']
training_metrics  = pd.DataFrame(columns = col_names)

In [12]:
### Set the path for the model

model_root_path = r'''/home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_'''

print(model_root_path)

/home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_


#### Model construction
###### Change the "range(10)" | test_size=0.2 | fit_model()

In [13]:
for i in range(len(paras)):
    optimizer, activation = paras[i].split(' ')[0], paras[i].split(' ')[1]
    name = 'DNN_'+'paras_'+str(i)+'_optimizer_'+optimizer+'_activation_'+activation
    print(name)
    for j in range(10):
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1, random_state=j)
        model_path = model_root_path + name +'_seed_'+ str(j)+ '_weights.h5'
        model = create_model(optimizer, activation)
        model.summary()
        fit_model(X_train, y_train, X_test, y_test,10,model_path,model)
        result = predict_validation(X_test, y_test, model_path, model, optimizer)
        print(result)
        training_metrics.loc[len(training_metrics)] = [str(j), name, result[0] , result[1], result[2], result[3],  result[4]]

training_metrics.to_csv(r'''results/dnn_SVR_fs_SARS_best_training_metrics_all.csv''')

DNN_paras_0_optimizer_Adam_activation_tanh
Model: "sequential_122"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_976 (Dense)            (None, 512)               26112     
_________________________________________________________________
dense_977 (Dense)            (None, 256)               131328    
_________________________________________________________________
dense_978 (Dense)            (None, 128)               32896     
_________________________________________________________________
dense_979 (Dense)            (None, 64)                8256      
_________________________________________________________________
dense_980 (Dense)            (None, 32)                2080      
_________________________________________________________________
dense_981 (Dense)            (None, 16)                528       
_________________________________________________________________
dense_982


Epoch 00001: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_2_weights.h5

Epoch 00002: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_2_weights.h5

Epoch 00003: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_2_weights.h5

Epoch 00004: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_2_weights.h5

Epoch 00005: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_2_weights.h5

Epoch 00006: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activati


Epoch 00003: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_4_weights.h5

Epoch 00004: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_4_weights.h5

Epoch 00005: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_4_weights.h5

Epoch 00006: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_4_weights.h5

Epoch 00007: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_4_weights.h5

Epoch 00008: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activati


Epoch 00005: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_6_weights.h5

Epoch 00006: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_6_weights.h5

Epoch 00007: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_6_weights.h5

Epoch 00008: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_6_weights.h5

Epoch 00009: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_6_weights.h5

Epoch 00010: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activati


Epoch 00007: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_8_weights.h5

Epoch 00008: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_8_weights.h5

Epoch 00009: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_8_weights.h5

Epoch 00010: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_0_optimizer_Adam_activation_tanh_seed_8_weights.h5
[5.816106295681099, 2.4116604851597785, -1.5802499083730934, -0.04213867612465272, 0.8016638031223716]
Model: "sequential_131"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1048 (Dense)           (None, 512)               26

...

[12.997586751884473, 3.605216602630759, -9.486309517501189, -0.03520003756136487, 0.8338206586765557]
DNN_paras_11_optimizer_RMSprop_activation_softmax
Model: "sequential_232"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1856 (Dense)           (None, 512)               26112     
_________________________________________________________________
dense_1857 (Dense)           (None, 256)               131328    
_________________________________________________________________
dense_1858 (Dense)           (None, 128)               32896     
_________________________________________________________________
dense_1859 (Dense)           (None, 64)                8256      
_________________________________________________________________
dense_1860 (Dense)           (None, 32)                2080      
_________________________________________________________________
dense_1861 (Dense)           (No


Epoch 00001: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_2_weights.h5

Epoch 00002: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_2_weights.h5

Epoch 00003: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_2_weights.h5

Epoch 00004: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_2_weights.h5

Epoch 00005: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_2_weights.h5

Epoch 00006: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_


Epoch 00003: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_4_weights.h5

Epoch 00004: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_4_weights.h5

Epoch 00005: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_4_weights.h5

Epoch 00006: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_4_weights.h5

Epoch 00007: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_4_weights.h5

Epoch 00008: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_


Epoch 00005: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_6_weights.h5

Epoch 00006: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_6_weights.h5

Epoch 00007: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_6_weights.h5

Epoch 00008: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_6_weights.h5

Epoch 00009: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_6_weights.h5

Epoch 00010: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_


Epoch 00007: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_8_weights.h5

Epoch 00008: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_8_weights.h5

Epoch 00009: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_8_weights.h5

Epoch 00010: saving model to /home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_11_optimizer_RMSprop_activation_softmax_seed_8_weights.h5
[33.20659534434693, 5.762516407295248, -13.731731202756462, nan, nan]
Model: "sequential_241"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1928 (Dense)           (None, 512)               26112  

## Optimized model

In [21]:
model = create_model('RMSprop','tanh')
### Load the optimized model weights

model_path = r'''/home/akanksha/Documents/IMTech/anti-corona/overall_data/models/best/best_model_DNN_paras_9_optimizer_RMSprop_activation_tanh_seed_6_weights.h5'''

model.summary()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=6)
### Optimized DNN model training performance
print_result('DNN', 'training', predict_validation(X_test, y_test, model_path, model, 'RMSprop'))
### Optimized DNN model testing performance
print_result('DNN', 'testing', predict_validation(X_val, y_val, model_path, model, 'RMSprop'))

Model: "sequential_249"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1992 (Dense)           (None, 512)               26112     
_________________________________________________________________
dense_1993 (Dense)           (None, 256)               131328    
_________________________________________________________________
dense_1994 (Dense)           (None, 128)               32896     
_________________________________________________________________
dense_1995 (Dense)           (None, 64)                8256      
_________________________________________________________________
dense_1996 (Dense)           (None, 32)                2080      
_________________________________________________________________
dense_1997 (Dense)           (None, 16)                528       
_________________________________________________________________
dense_1998 (Dense)           (None, 8)              